In [1]:
data_partition = {
        "gwilliams2023": {
            "testing_subjects": [19, 20, 21],
            "testing_tasks": [0],
        },
        "schoffelen2022": {
            "testing_subjects": [],
            "testing_tasks": [8, 9],
        },
}

In [2]:
# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=None
#     )

In [3]:
from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=512,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=3,
    layout_proj=False,
    layout_scaling="midpoint",
    # Merger with spatial attn
    merger=True,
    merger_emb_type="linear",
    merger_emb_dim=512,
    merger_channels=256,
    merger_dropout=True,
    merger_conditional=None,
    # Inital
    initial_linear=256,
    initial_depth=1,
    # Conditional layers
    conditional_layers=True,
    conditional_layers_dim='input',  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

In [4]:
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=100,
    frequency_bands={'all': (0.5, 80)},
    max_random_shift=1.0,
    window_size= 4,
    window_stride=1,
    brain_clipping=20,
    baseline_window=None,
    notch_filter=True,
    scaling='both',
    delay=0.15,
    # Hyperparameters
    learning_rate = 5e-4,
    weight_decay = 1e-4,
    epochs = 50,
    batch_size = 256,
    use_clip_loss = True,
    use_mse_loss = True,
    alpha =0.6,
    random_test_size =20,
    seed = 42,
)

session = TrainingSessionV0(
    config=config,
    studies={study: 'audio' for study in data_partition.keys()},
    data_path='data',
    save_path='saves/Dec18_CLIP_MSE_full_both_datasets',
    clear_cache=False,
)

Loading Gwilliams2023 with batch type audio
Loading Schoffelen2022 with batch type audio
Data partitioned on studies ['gwilliams2023', 'schoffelen2022'].
Train: 165, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

Conditional layer study initialized with 3 conditions
Conditional layer subject initialized with 31 conditions
SimpleConv initialized with 14632832 parameters, cond: ['study', 'subject']
Merger True, merger channels 256
ConvBlocks: 6, hidden_dim: 512, params 13776896


In [ ]:
try:
    session.train(
        device='cuda',
        buffer_size=30,
        num_workers=multiprocessing.cpu_count() - 2,
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print('Exited')

2024-12-19 10:31:03,190	INFO worker.py:1821 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 165/165 [05:18<00:00,  1.93s/it]


Testing at epoch 1
Test unseen_subject completed. Accuracy: 0.0060, Top 1: 0.0094, Top 5: 0.0464, Top 10: 0.0920, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0061, Top 1: 0.0094, Top 5: 0.0450, Top 10: 0.0930, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0062, Top 1: 0.0123, Top 5: 0.0517, Top 10: 0.0873, Perplexity: 0.0000
Epoch 1 completed in 5.91m. 0.04m per recording.


Training Epoch 2: 100%|██████████| 165/165 [05:17<00:00,  1.92s/it]


Testing at epoch 2
Test unseen_subject completed. Accuracy: 0.0052, Top 1: 0.0091, Top 5: 0.0486, Top 10: 0.0986, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0058, Top 1: 0.0089, Top 5: 0.0467, Top 10: 0.0983, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0049, Top 1: 0.0074, Top 5: 0.0492, Top 10: 0.0960, Perplexity: 0.0000
Epoch 2 completed in 5.87m. 0.04m per recording.


Training Epoch 3: 100%|██████████| 165/165 [05:12<00:00,  1.89s/it]


...

Training Epoch 13: 100%|██████████| 165/165 [05:12<00:00,  1.90s/it]


Testing at epoch 13
Test unseen_subject completed. Accuracy: 0.0040, Top 1: 0.0089, Top 5: 0.0577, Top 10: 0.1187, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0039, Top 1: 0.0076, Top 5: 0.0485, Top 10: 0.0961, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0086, Top 1: 0.0185, Top 5: 0.0542, Top 10: 0.1034, Perplexity: 0.0000
Epoch 13 completed in 5.76m. 0.03m per recording.


Training Epoch 14: 100%|██████████| 165/165 [05:16<00:00,  1.92s/it]


Testing at epoch 14
Test unseen_subject completed. Accuracy: 0.0050, Top 1: 0.0094, Top 5: 0.0511, Top 10: 0.1066, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0059, Top 1: 0.0100, Top 5: 0.0593, Top 10: 0.1134, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0074, Top 1: 0.0148, Top 5: 0.0480, Top 10: 0.0910, Perplexity: 0.0000
Epoch 14 completed in 5.83m. 0.04m per recording.


Training Epoch 15:  22%|██▏       | 36/165 [01:26<03:30,  1.63s/it] Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x758750138e10>>
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/brain/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Training Epoch 15:  25%|██▍       | 41/165 [01:37<05:14,  2.54s/it]

In [ ]:
# import os

# def delete_file_recursive(directory, filename):
#     for root, dirs, files in os.walk(directory):
#         if filename in files:
#             file_path = os.path.join(root, filename)
#             try:
#                 os.remove(file_path)
#                 print(f"Deleted: {file_path}")
#             except PermissionError:
#                 print(f"Permission denied: {file_path}")
#             except Exception as e:
#                 print(f"Error deleting {file_path}: {e}")

# directory = os.getcwd()
# filename = 'training_log.log'
# delete_file_recursive(directory, filename)